In [ ]:
# Import our packages for numerical computing
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.linalg as sla
%matplotlib inline

__Demonstrate $QR$ factorization__

- Use Gram-Schmit to form reduced $QR$
  $$ A = \hat{Q} \hat{R} $$


In [ ]:
# Generate a random matrix for the QR 
A = sp.rand(4,4)

# Define function for classical Gram-Schmidt
def classical_GS(A):
    '''
    Carry out classical Gram-Schmidt on A
    Return reduced Q R
    '''
    m = A.shape[0]
    n = A.shape[1]
    
    # Preallocate Q and R (more efficient)
    Q = np.zeros( (m,n) )
    R = np.zeros( (n,n) )
    
    # Loop over columns of A
    for j in range(n):
        # Create orthonormal qj based on column j of A
        vj = A[:,j]
        
        # Orthogonalize vj w.r.t. all the previous columns of A
        #  Note: Python will not execute this loop for the i==j case 
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], A[:,j])
            vj = vj - R[i,j]*Q[:,i]
        
        # Set column j in Q
        R[j,j] = sla.norm(vj)
        Q[:,j] = vj/R[j,j]
    ## End loops    
    return (Q,R)
    
np.set_printoptions(precision=3, suppress=True)
[Q,R] = classical_GS(A)
print(Q)
print("----")
print(R)

__Let's verify some basic properties of $QR$__

In [ ]:
# Is A == QR ?
print(A - np.dot(Q,R))
print("------")

# Is Q* Q = I ?
print(np.dot(Q.T,Q))

#


__Let's solve a linear system with $QR$__

In [ ]:
b = sp.rand(4,1)
Qb = np.dot(Q.T, b)
x = sla.solve_triangular(R, Qb)
print(b)
print('-------')
print(np.dot(A,x))
print('-------')
print(b - np.dot(A,x))

__What if $A$ is singular?__

In [ ]:
# Let's make column 2 a multiple of column 1
# Predictions on what will happen? 
A[:,2] = 1.23*A[:,1]

In [ ]:
[Q,R] = classical_GS(A)
print(Q)
print("----")
print(R)
print R[2,2]

In [ ]:
# Now let's solve a linear system with this Q and R
b = sp.rand(4,1)
Qb = np.dot(Q.T, b)
x = sla.solve_triangular(R, Qb)
print(b)
print('-------')
print(np.dot(A,x))
print('-------')
print(b - np.dot(A,x))

__Oh no!  This tiny entry is now messing everything up.__
- Problem is that **cond(R)** is huge.  
- Let's change the algorithm to set this numerically zero value to 0.
- Note the zero check !!

In [ ]:
# Define function for classical Gram-Schmidt
def classical_GS_new(A):
    '''
    Carry out classical Gram-Schmidt on A
    Return reduced Q R
    
    UPDATE: Added check for linear dependence (singularness)
    '''
    m = A.shape[0]
    n = A.shape[1]
    
    # Preallocate Q and R (more efficient)
    Q = np.zeros( (m,n) )
    R = np.zeros( (n,n) )
    
    # Loop over columns of A
    for j in range(n):
        # Create orthonormal qj based on column j of A
        vj = A[:,j]
        
        # Orthogonalize vj w.r.t. all the previous columns of A
        #  Note: Python will not execute this loop for the i==j case 
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], A[:,j])
            vj = vj - R[i,j]*Q[:,i]
        
        # Set column j in Q
        R[j,j] = sla.norm(vj)
        if R[j,j] < 1e-10*R[0,0]: # NEW CHECK for linear dependence
            R[j,j] = 0.0
            Q[:,j] = 0.0
        else:
            Q[:,j] = vj/R[j,j]
            
    ## End loops    
    return (Q,R)

In [ ]:
# Now lets use the new QR 
[Q,R] = classical_GS_new(A)

print(Q)
print(R)

__Now, the QR algorithm is a bit more robust, warning us when we have a numerically singular A (at least in some cases)__
- But we still can't invert $R$

__Thoughts for home__
- What if $A$ is complex valued?
- What do we need to change above?
- Does the order of any dot products matter?
